In [539]:
import pandas as pd
import os

In [540]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('alba_mf.xlsm')))
DATA_RAW = os.path.join(BASE_DIR, 'data', 'raw')

In [ ]:
excel_path = os.path.join(DATA_RAW, 'alba_mf.xlsm')
df = pd.read_excel(excel_path, sheet_name='EC DATA')

In [ ]:
alba_mf_path = os.path.join(DATA_RAW, 'alba_mf.csv')
df.to_csv(alba_mf_path, index=False)

In [542]:
print(df.iloc[1])

Unnamed: 0                             NaN
Unnamed: 1                             NaT
Unnamed: 2                          Butane
Unnamed: 3                      Condensate
Unnamed: 4               Diesel Production
                          ...             
Unnamed: 92              Propane Flare Gas
Unnamed: 93    Residue Compressor Fuel Gas
Unnamed: 94                   TG3 Fuel Gas
Unnamed: 95                   TG4 Fuel Gas
Unnamed: 96              Total HP Fuel Gas
Name: 1, Length: 97, dtype: object


In [543]:
# Obtener los nombres de columna de la segunda fila (índice 1)
column_names = df.iloc[1].values.tolist()

# Renombrar la segunda columna a 'datetime'
columns_to_rename = [1, 18, 35, 47, 55]

# Convertir todos los nombres de columnas a strings primero
column_names = [str(col) for col in column_names]

# Ahora aplicar la transformación a minúsculas y guiones bajos
column_names = [col.lower().replace(' ', '_').replace('-', '_') for col in column_names]

for i, pos in enumerate(columns_to_rename):
    column_names[pos] = f'date_{i}'

# Asignar los nuevos nombres de columna
df.columns = column_names

# Eliminar las dos primeras filas (índices 0 y 1) y restablecer índices
df = df.drop(df.index[:2]).reset_index(drop=True)
print(df)


      nan     date_0       butane    condensate diesel_production  \
0     NaN 2025-05-12  3727.551941  14172.638778          40.95846   
1     NaN 2025-05-11  3495.934299  14222.852652         44.237979   
2     NaN 2025-05-10  3278.857892  14277.377068         43.081713   
3     NaN 2025-05-09  3483.932484  13774.898757         31.531699   
4     NaN 2025-05-08  3241.778655  14050.753548          32.66754   
...   ...        ...          ...           ...               ...   
6469  NaN        NaT          NaN           NaN               NaN   
6470  NaN        NaT          NaN           NaN               NaN   
6471  NaN        NaT          NaN           NaN               NaN   
6472  NaN        NaT          NaN           NaN               NaN   
6473  NaN        NaT          NaN           NaN               NaN   

     eglng_propane_sales llc_share_of_secondary_condensate lpg_sonagas  \
0                    NaN                       2965.753092  198.985714   
1                    Na

In [544]:
df = df.iloc[:, 1:50]

In [545]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6474 entries, 0 to 6473
Data columns (total 49 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   date_0                                498 non-null    datetime64[ns]
 1   butane                                498 non-null    object        
 2   condensate                            498 non-null    object        
 3   diesel_production                     498 non-null    object        
 4   eglng_propane_sales                   2 non-null      object        
 5   llc_share_of_secondary_condensate     498 non-null    object        
 6   lpg_sonagas                           498 non-null    object        
 7   primary_condensate_production         498 non-null    object        
 8   propane                               498 non-null    object        
 9   psc_share_of_secondary_condensate     498 non-null    object        
 10  

Si por alguna razón necesitas ajustar esto aún más porque hay más filas antes de los datos reales, puedes hacer:
``py
# Suponiendo que quieres usar la fila en el índice n como encabezados
n = 1  # Este es el índice para la fila 2
df.columns = df.iloc[n]

# Eliminar todas las filas hasta la fila de encabezados inclusive
df = df.drop(df.index[:n+1]).reset_index(drop=True)
``
El método df.index[:n+1] crea un slice de índices desde 0 hasta n inclusive, lo que te permite eliminar todas esas filas de una vez.

In [546]:
date_columns = [col for col in df.columns if 'date' in str(col).lower()]
print(date_columns)

['date_0', 'date_1', 'date_2', 'date_3']


In [547]:
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce').dt.floor('D')
    print(df[col].dtype)

datetime64[ns]
datetime64[ns]
datetime64[ns]
datetime64[ns]


In [548]:
print(df.dtypes)

date_0                                  datetime64[ns]
butane                                          object
condensate                                      object
diesel_production                               object
eglng_propane_sales                             object
llc_share_of_secondary_condensate               object
lpg_sonagas                                     object
primary_condensate_production                   object
propane                                         object
psc_share_of_secondary_condensate               object
secondary_condensate                            object
total_liquid_hydrocarbons_production            object
nan                                            float64
nan                                            float64
nan                                            float64
nan                                            float64
nan                                            float64
date_1                                  datetime64[ns]
ampco_gas_

In [549]:
# Encontrar columnas que literalmente se llaman "nan" (como texto)
columnas_nan = [col for col in df.columns if col == "nan"]
print(f"Columnas con nombre 'nan': {len(columnas_nan)}")

# Eliminar estas columnas
if columnas_nan:
    df = df.drop(columns=columnas_nan)
    print(f"Se eliminaron {len(columnas_nan)} columnas llamadas 'nan'")

Columnas con nombre 'nan': 11
Se eliminaron 11 columnas llamadas 'nan'


In [550]:
columns_to_exclude = date_columns + ['tank_name', 'product']
print(df.columns)

Index(['date_0', 'butane', 'condensate', 'diesel_production',
       'eglng_propane_sales', 'llc_share_of_secondary_condensate',
       'lpg_sonagas', 'primary_condensate_production', 'propane',
       'psc_share_of_secondary_condensate', 'secondary_condensate',
       'total_liquid_hydrocarbons_production', 'date_1', 'ampco_gas_sales',
       'ampco_hp_sales_gas', 'ampco_lp_sales_gas', 'ampco_regen_sales_gas',
       'eglng_feed_sales_gas', 'eglng_fuel_sales_gas', 'eglng_gas_sales',
       'eg_power_plant_fuel_gas', 'gas_sales', 'htr1_and_htr2_fuel_gas',
       'offshore_gas', 'propane_flare_gas', 'residue_compressor_fuel_gas',
       'tg3_and_tg4_fuel_gas', 'date_2', 'tank_name', 'liquid_dip_level',
       'average_temperature_(degf)', 'volume_correction_factor',
       'gross_oil_volume_(bbls)', 'standard_gross_oil_volume_(bbls)',
       'standard_net_oil_volume_(bbls)', 'date_3', 'product',
       'bill_of_lading_volume_(bbls)'],
      dtype='object')


In [551]:
for col in df.columns:
    if col not in columns_to_exclude:
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(df.dtypes)

date_0                                  datetime64[ns]
butane                                         float64
condensate                                     float64
diesel_production                              float64
eglng_propane_sales                            float64
llc_share_of_secondary_condensate              float64
lpg_sonagas                                    float64
primary_condensate_production                  float64
propane                                        float64
psc_share_of_secondary_condensate              float64
secondary_condensate                           float64
total_liquid_hydrocarbons_production           float64
date_1                                  datetime64[ns]
ampco_gas_sales                                float64
ampco_hp_sales_gas                             float64
ampco_lp_sales_gas                             float64
ampco_regen_sales_gas                          float64
eglng_feed_sales_gas                           float64
eglng_fuel

In [552]:
# Separar en diferentes df
df_liquid_hydrocarbons_cached = df.iloc[:, 0:11]

In [553]:
print(df_liquid_hydrocarbons_cached.head())

      date_0       butane    condensate  diesel_production  \
0 2025-05-12  3727.551941  14172.638778          40.958460   
1 2025-05-11  3495.934299  14222.852652          44.237979   
2 2025-05-10  3278.857892  14277.377068          43.081713   
3 2025-05-09  3483.932484  13774.898757          31.531699   
4 2025-05-08  3241.778655  14050.753548          32.667540   

   eglng_propane_sales  llc_share_of_secondary_condensate  lpg_sonagas  \
0                  NaN                        2965.753092   198.985714   
1                  NaN                        2918.003607   206.383333   
2                  NaN                        2922.824261   265.528571   
3                  NaN                        2825.651675   257.840476   
4                  NaN                        2812.694846   206.816667   

   primary_condensate_production      propane  \
0                   10838.885686  5219.833296   
1                   10936.849045  5454.296950   
2                   10986.552807  5

In [554]:
df_gas_production = df.iloc[:, 12:24]

In [555]:
print(df_gas_production.head())

      date_1  ampco_gas_sales  ampco_hp_sales_gas  ampco_lp_sales_gas  \
0 2025-05-12         1.419290                 0.0            1.419290   
1 2025-05-11         1.403316                 0.0            1.403316   
2 2025-05-10         1.400136                 0.0            1.400136   
3 2025-05-09         1.408744                 0.0            1.408744   
4 2025-05-08         1.408744                 0.0            1.408744   

   ampco_regen_sales_gas  eglng_feed_sales_gas  eglng_fuel_sales_gas  \
0                    0.0                   NaN                   NaN   
1                    0.0                   NaN                   NaN   
2                    0.0                   NaN                   NaN   
3                    0.0                   NaN                   NaN   
4                    0.0                   NaN                   NaN   

   eglng_gas_sales  eg_power_plant_fuel_gas  gas_sales  \
0              NaN                40.803421  66.041843   
1           

In [556]:
df_tank_data = df.iloc[:, 27:35]

In [557]:
print(df_tank_data.head())

      date_2             tank_name  liquid_dip_level  \
0 2025-05-12      Butane Tank 4500         69.842520   
1 2025-05-12       Butane Tank 761        827.152771   
2 2025-05-12  Condensate Tank 4410        658.443100   
3 2025-05-12  Condensate Tank 4420        478.995713   
4 2025-05-12  Condensate Tank 4450          0.000000   

   average_temperature_(degf)  volume_correction_factor  \
0                   25.879999                    1.0340   
1                   25.879999                    1.0340   
2                   86.000000                    0.9836   
3                   86.000000                    0.9836   
4                   86.000000                    0.9836   

   gross_oil_volume_(bbls)  standard_gross_oil_volume_(bbls)  \
0              8810.969142                       9110.542093   
1            104320.507504                     107867.404759   
2            307933.744290                     302883.630884   
3            223340.296558                     21967

In [558]:
df_daily_lifting_data = df.iloc[:, 35:38]

In [559]:
print(df_daily_lifting_data.head())

      date_3 product  bill_of_lading_volume_(bbls)
0 2025-05-12  DIESEL                           NaN
1 2025-05-12     LPG                           NaN
2 2025-05-11     LPG                           NaN
3 2025-05-10     LPG                           NaN
4 2025-05-09     LPG                           NaN


In [560]:
# df_liquid_hydrocarbons_cached = df_liquid_hydrocarbons_cached.drop(df_liquid_hydrocarbons_cached.columns[[0, 5, 6, 10, 12]], axis=1)

In [561]:
# Configurar pandas para mostrar todos los registros (desactivando la limitación por defecto)
pd.set_option('display.max_rows', None)

# Mostrar el DataFrame completo
print(df_liquid_hydrocarbons_cached)

# Para restablecer la configuración posteriormente (opcional)
pd.reset_option('display.max_rows')

         date_0       butane    condensate  diesel_production  \
0    2025-05-12  3727.551941  14172.638778          40.958460   
1    2025-05-11  3495.934299  14222.852652          44.237979   
2    2025-05-10  3278.857892  14277.377068          43.081713   
3    2025-05-09  3483.932484  13774.898757          31.531699   
4    2025-05-08  3241.778655  14050.753548          32.667540   
5    2025-05-07  3481.619170  14372.423955          34.602868   
6    2025-05-06  3162.829751  14364.519836          19.715514   
7    2025-05-05  2163.969467  13812.758866          41.587130   
8    2025-05-04  3258.814118  15114.759250          42.235379   
9    2025-05-03  1625.366413  12660.429592           8.567399   
10   2025-05-02  3036.195310   9821.476006          27.993762   
11   2025-05-01   177.318007   3321.107468          -0.089010   
12   2025-04-30     0.253649      0.106792           0.000000   
13   2025-04-29     0.006501   3510.098400          13.060790   
14   2025-04-28  3424.523

In [562]:
# Elimina filas solo si todos sus valores son NaN
df_liquid_hydrocarbons_cached = df_liquid_hydrocarbons_cached.dropna(how='all')

In [563]:
print(df_liquid_hydrocarbons_cached.shape)
print(df_liquid_hydrocarbons_cached.info())

(498, 11)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 497
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   date_0                             498 non-null    datetime64[ns]
 1   butane                             498 non-null    float64       
 2   condensate                         498 non-null    float64       
 3   diesel_production                  498 non-null    float64       
 4   eglng_propane_sales                2 non-null      float64       
 5   llc_share_of_secondary_condensate  498 non-null    float64       
 6   lpg_sonagas                        498 non-null    float64       
 7   primary_condensate_production      498 non-null    float64       
 8   propane                            498 non-null    float64       
 9   psc_share_of_secondary_condensate  498 non-null    float64       
 10  secondary_condensate        

In [564]:
lhc_columns_to_delete = ['eglng_propane_sales', 'llc_share_of_secondary_condensate', 'psc_share_of_secondary_condensate']

In [565]:
df_liquid_hydrocarbons_cached = df_liquid_hydrocarbons_cached.drop(columns=lhc_columns_to_delete)
df_liquid_hydrocarbons_cached = df_liquid_hydrocarbons_cached.rename(columns={'date_0': 'date'})

In [566]:
print(df_liquid_hydrocarbons_cached.head())

        date       butane    condensate  diesel_production  lpg_sonagas  \
0 2025-05-12  3727.551941  14172.638778          40.958460   198.985714   
1 2025-05-11  3495.934299  14222.852652          44.237979   206.383333   
2 2025-05-10  3278.857892  14277.377068          43.081713   265.528571   
3 2025-05-09  3483.932484  13774.898757          31.531699   257.840476   
4 2025-05-08  3241.778655  14050.753548          32.667540   206.816667   

   primary_condensate_production      propane  secondary_condensate  
0                   10838.885686  5219.833296           3333.753092  
1                   10936.849045  5454.296950           3286.003607  
2                   10986.552807  5790.175738           3290.824261  
3                   10581.247083  5127.037318           3193.651675  
4                   10870.058702  5203.358145           3180.694846  


In [ ]:
df_gas_production = df_gas_production.dropna(how='all')

In [568]:
print(df_gas_production.shape)
print(df_gas_production.info())

(498, 12)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 497
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   date_1                   498 non-null    datetime64[ns]
 1   ampco_gas_sales          498 non-null    float64       
 2   ampco_hp_sales_gas       498 non-null    float64       
 3   ampco_lp_sales_gas       498 non-null    float64       
 4   ampco_regen_sales_gas    498 non-null    float64       
 5   eglng_feed_sales_gas     0 non-null      float64       
 6   eglng_fuel_sales_gas     0 non-null      float64       
 7   eglng_gas_sales          152 non-null    float64       
 8   eg_power_plant_fuel_gas  498 non-null    float64       
 9   gas_sales                498 non-null    float64       
 10  htr1_and_htr2_fuel_gas   498 non-null    float64       
 11  offshore_gas             498 non-null    float64       
dtypes: datetime64[ns](1), floa

In [573]:
gp_columns = ['date_1', 'ampco_gas_sales', 'eglng_gas_sales', 'gas_sales', 'offshore_gas']
df_gas_production = df_gas_production[gp_columns]
df_gas_production = df_gas_production.rename(columns={'date_1': 'date'})

In [574]:
print(df_gas_production.head())

        date  ampco_gas_sales  eglng_gas_sales  gas_sales  offshore_gas
0 2025-05-12         1.419290              NaN  66.041843    352.125422
1 2025-05-11         1.403316              NaN  62.275150    352.411255
2 2025-05-10         1.400136              NaN  62.281059    353.302860
3 2025-05-09         1.408744              NaN  62.050750    348.031556
4 2025-05-08         1.408744              NaN  62.979855    340.288209


In [575]:
df_tank_data = df_tank_data.dropna(how='all')

In [577]:
print(df_tank_data.shape)
df_tank_data.info()

(6474, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6474 entries, 0 to 6473
Data columns (total 8 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date_2                            6474 non-null   datetime64[ns]
 1   tank_name                         6474 non-null   object        
 2   liquid_dip_level                  6398 non-null   float64       
 3   average_temperature_(degf)        5339 non-null   float64       
 4   volume_correction_factor          6471 non-null   float64       
 5   gross_oil_volume_(bbls)           5478 non-null   float64       
 6   standard_gross_oil_volume_(bbls)  5475 non-null   float64       
 7   standard_net_oil_volume_(bbls)    5475 non-null   float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 404.8+ KB


In [578]:
td_columns = ['date_2', 'tank_name', 'standard_net_oil_volume_(bbls)']
df_tank_data = df_tank_data[td_columns]
df_tank_data = df_tank_data.rename(columns={'date_2': 'date'})

In [579]:
print(df_tank_data.head())

        date             tank_name  standard_net_oil_volume_(bbls)
0 2025-05-12      Butane Tank 4500                     9110.542093
1 2025-05-12       Butane Tank 761                   107867.404759
2 2025-05-12  Condensate Tank 4410                   302883.630884
3 2025-05-12  Condensate Tank 4420                   219677.515695
4 2025-05-12  Condensate Tank 4450                        0.000000


In [580]:
df_daily_lifting_data = df_daily_lifting_data.dropna(how='all')

In [581]:
print(df_daily_lifting_data.shape)
df_daily_lifting_data.info()

(816, 3)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 816 entries, 0 to 815
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date_3                        815 non-null    datetime64[ns]
 1   product                       816 non-null    object        
 2   bill_of_lading_volume_(bbls)  76 non-null     float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 25.5+ KB


In [ ]:
df_daily_lifting_data = df_daily_lifting_data.rename(columns={'date_3': 'date'})